In [1]:
import pickle
import numpy as np
import tensorflow as tf

import sys
sys.path.append('/Users/xiejunyi/Dropbox/Research/MultiLevelOptNN/util')
sys.dont_write_bytecode = True

In [2]:
with open('lidc_data.pkl', 'rb') as f:
    data = pickle.load(f)

print("data loaded")

data loaded


In [3]:
train_img = data[0][0]
print("training img:")
print(type(train_img), train_img.shape)

train_label = data[0][1]
print("training label:")
print(type(train_label), train_label.shape)

val_img = data[1][0]
print("validation img:")
print(type(val_img), val_img.shape)

val_label = data[1][1]
print("validation label:")
print(type(val_label), val_label.shape)

test_img = data[2][0]
print("test img:")
print(type(test_img), test_img.shape)

test_label = data[2][1]
print("test label:")
print(type(test_label), test_label.shape)

training img:
(<type 'numpy.ndarray'>, (22000, 4096))
training label:
(<type 'numpy.ndarray'>, (22000,))
validation img:
(<type 'numpy.ndarray'>, (1004, 4096))
validation label:
(<type 'numpy.ndarray'>, (1004,))
test img:
(<type 'numpy.ndarray'>, (953, 4096))
test label:
(<type 'numpy.ndarray'>, (953,))


In [4]:
# hyper parameters
learning_rate = 0.0003
training_iters = 100000
batch_size = 128
display_step = 10

In [5]:
n_input = 4096
n_classes = 2
dropout = 0.75

In [6]:
# transform labels from number to one hot format
num_train = train_img.shape[0]
num_val = val_img.shape[0]
num_test = test_img.shape[0]

train_label_onehot = np.zeros([num_train, n_classes])
for idx in range(num_train):
    train_label_onehot[idx, train_label[idx]] = 1

val_label_onehot = np.zeros([num_val, n_classes])
for idx in range(num_val):
    val_label_onehot[idx, val_label[idx]] = 1
    
test_label_onehot = np.zeros([num_test, n_classes])
for idx in range(num_test):
    test_label_onehot[idx, test_label[idx]] = 1

In [7]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [8]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])), 
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])), 
    # fully connected, 16*16*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([16*16*64, 1024])), 
    # 1024 inputs, 2 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes])) 
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [9]:
def conv_net_graph(_X, _weights, _biases, _dropout):
    _X = tf.reshape(_X, shape=[-1, 64, 64, 1])
    conv1 = tf.nn.conv2d(_X, _weights['wc1'], strides=[1,1,1,1], padding='SAME', name='conv1')
    bias1 = tf.nn.bias_add(conv1, _biases['bc1'], name='bias1')
    relu1 = tf.nn.relu(bias1, name='relu1')
    pool1 = tf.nn.max_pool(relu1, ksize=[1,2,2,1], strides = [1,2,2,1], padding='SAME', name='pool1')
    drop1 = tf.nn.dropout(pool1, _dropout, name='drop1')
    
    conv2 = tf.nn.conv2d(drop1, _weights['wc2'], strides=[1,1,1,1], padding='SAME', name='conv2')
    bias2 = tf.nn.bias_add(conv2, _biases['bc2'], name='bias2')
    relu2 = tf.nn.relu(bias2, name='relu2')
    pool2 = tf.nn.max_pool(relu2, ksize=[1,2,2,1], strides = [1,2,2,1], padding='SAME', name='pool2')
    drop2 = tf.nn.dropout(pool2, _dropout, name='drop2')
    
    drop2_flatten = tf.reshape(drop2, [-1, _weights['wd1'].get_shape().as_list()[0]])
    
    fc3   = tf.matmul(drop2_flatten, _weights['wd1'], name='fc3')
    bias3 = tf.add(fc3, _biases['bd1'])
    relu3 = tf.nn.relu(bias3, name='relu3')
    drop3 = tf.nn.dropout(relu3, _dropout, name='drop3')
    
    fc4   = tf.matmul(drop3, _weights['out'], name='fc4')
    bias4 = tf.add(fc4, _biases['out'], name='bias4')
    
    out = bias4
    return out

In [10]:
# Construct model
pred = conv_net_graph(x, weights, biases, keep_prob)
#print(type(pred))

In [11]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [12]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [13]:
# Initializing the variables
init = tf.initialize_all_variables()

In [14]:
def next_batch(batch_size, input_samples, input_labels, n_classes, fake_data=False):
    index_in_epoch = 0
    num_examples = input_samples.shape[0]
    
    if fake_data:
        fake_image = [1.0 for _ in xrange(4096)]
        fake_label = 0
        return [fake_image for _ in xrange(batch_size)], [fake_label for _ in xrange(batch_size)]
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        # Finished epoch
        epochs_completed += 1
        # Shuffle the data
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        input_samples = input_samples[perm]
        input_labels = input_labels[perm]
        # start next epoch
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    output_samples = input_samples[start:end]

    #output_labels = np.zeros([end-start, n_classes])
    #original_labels = input_labels[start:end]
    #
    #for idx in range(batch_size):
    #    output_labels[idx, original_labels[idx]] = 1
    
    output_labels = input_labels[start:end]
    return output_samples, output_labels

In [15]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_xs, batch_ys = next_batch(batch_size, train_img, train_label_onehot, n_classes)
        
        #print(batch_xs.shape)
        #print(batch_ys.shape)
        sess.run(optimizer, feed_dict={x:batch_xs, y:batch_ys, keep_prob:dropout})
        if step % display_step == 0:
            # Calculate batch accuracy
            train_acc = sess.run(accuracy, feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(train_acc)
            
            val_acc = sess.run(accuracy, feed_dict={x: val_img, y: val_label_onehot, keep_prob:1.})
            print "Validation Accuracy= " + "{:.5f}".format(val_acc)
            print "Testing Accuracy= ", sess.run(accuracy, feed_dict={x: test_img, y: test_label_onehot, keep_prob:1.})        
        step += 1
    print "Optimization Finished!"

Iter 1280, Minibatch Loss= 4739.715332, Training Accuracy= 0.69531
Validation Accuracy= 0.61653
Testing Accuracy=  0.63064
Iter 2560, Minibatch Loss= 12460.481445, Training Accuracy= 0.64062
Validation Accuracy= 0.58865
Testing Accuracy=  0.566632
Iter 3840, Minibatch Loss= 9848.937500, Training Accuracy= 0.67188
Validation Accuracy= 0.62649
Testing Accuracy=  0.632739
Iter 5120, Minibatch Loss= 9481.997070, Training Accuracy= 0.67969
Validation Accuracy= 0.65040
Testing Accuracy=  0.652676
Iter 6400, Minibatch Loss= 7892.372070, Training Accuracy= 0.71094
Validation Accuracy= 0.67530
Testing Accuracy=  0.70829
Iter 7680, Minibatch Loss= 6804.294434, Training Accuracy= 0.73438
Validation Accuracy= 0.69821
Testing Accuracy=  0.731375
Iter 8960, Minibatch Loss= 5066.767578, Training Accuracy= 0.76562
Validation Accuracy= 0.74402
Testing Accuracy=  0.749213
Iter 10240, Minibatch Loss= 4584.107910, Training Accuracy= 0.80469
Validation Accuracy= 0.76096
Testing Accuracy=  0.761805
Iter 115

KeyboardInterrupt: 